In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



In [2]:
ls -1 ${FD_RES}/results/region

annotation_ccre
annotation_chipseq_histone
annotation_chipseq_tf
annotation_chipseq_tf_subset
annotation_chromHMM
annotation_crispri_hcrff
annotation_enhancer_junke
annotation_enhancer_zscore_junke
annotation_gencode
annotation_hic
annotation_tss_pol2
annotation_tss_pol2_span
hic_intact_ENCSR479XDG
KS91_K562_ASTARRseq_peak_macs_input
region_screened_crispri_flowfish.bed
region_screened_selected.bed
region_screened_selected_perbase.bed
region_screened_selected_perbase.bed.gz
region_screened_tmpra.bed
region_screened_tmpra_crispr.bed


In [7]:
ls -1 ${FD_RES}/results/region/annotation_tss_pol2

description.tsv
K562.TSS.selected_by_highest_Pol2_signal.bed
K562.TSS.selected_by_highest_Pol2_signal.bed.gz


In [9]:
FDIRY=${FD_RES}/results/region/annotation_tss_pol2
FNAME=K562.TSS.selected_by_highest_Pol2_signal.bed.gz
FPATH=${FDIRY}/${FNAME}
zcat ${FPATH} | head

chr1	11873	11874	DDX11L1	2.3e-4
chr1	17436	17437	MIR6859-1	9.43812
chr1	17436	17437	MIR6859-2	9.43812
chr1	17436	17437	MIR6859-3	9.43812
chr1	17436	17437	MIR6859-4	9.43812
chr1	29370	29371	WASH7P	2.3e-4
chr1	30365	30366	MIR1302-10	0.562995
chr1	30365	30366	MIR1302-11	0.562995
chr1	30365	30366	MIR1302-2	0.562995
chr1	30365	30366	MIR1302-9	0.562995


In [10]:
ls -1 ${FD_RES}/results/region/hic_intact_ENCSR479XDG/loop

annotation_enhancer_zscore_junke
annotation_tss_pol2
loopA.bed.gz
loopB.bed.gz


**Script**

In [31]:
cat > run_annot_test.sh << 'EOF'
#!/bin/bash

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### setup input and output
source ../config/config_duke.sh
FP_REG=$1
FP_ANT=$2
FD_OUT=$3

### show I/O file
echo "Input: " ${FP_REG}
echo
echo "show first few lines of input"
zcat ${FP_REG} | head -5
echo
echo "Input: " ${FP_ANT}
echo
echo "show first few lines of input"
zcat ${FP_ANT} | head -5
echo

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

chmod +x run_annot_test.sh

In [38]:
cat > run_annot.sh << 'EOF'
#!/bin/bash

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### setup input and output
source ../config/config_duke.sh
FP_REG=$1
FP_ANT=$2
FP_OUT=$3

### show I/O file
echo "Input: " ${FP_REG}
echo
echo "show first few lines of input"
zcat ${FP_REG} | head -5
echo
echo "Input: " ${FP_ANT}
echo
echo "show first few lines of input"
zcat ${FP_ANT} | head -5
echo

### execute: annotation using intersect
bedtools intersect \
    -a ${FP_REG} \
    -b ${FP_ANT} \
    -wo \
    -f 0.7 \
    -F 0.7 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

chmod +x run_annot.sh

**RUN**

In [33]:
FD_REG=${FD_RES}/results/region/hic_intact_ENCSR479XDG/loop
FD_ANT=${FD_RES}/results/region/annotation_tss_pol2
FD_OUT=${FD_RES}/results/region/hic_intact_ENCSR479XDG/loop/annotation_tss_pol2

for PREFIX in "loopA" "loopB"; do
    echo ${PREFIX}
    FN_REG=${PREFIX}.bed.gz
    FN_ANT=K562.TSS.selected_by_highest_Pol2_signal.bed.gz
    FN_OUT=${PREFIX}.annotation.${FN_ANT}
    
    FP_REG=${FD_REG}/${FN_REG}
    FP_ANT=${FD_ANT}/${FN_ANT}
    FP_OUT=${FD_OUT}/${FN_OUT}
    
    echo ${FN_REG}
    echo ${FN_ANT}
    echo ${FN_OUT}
    echo
done

loopA
loopA.bed.gz
K562.TSS.selected_by_highest_Pol2_signal.bed.gz
loopA.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed.gz

loopB
loopB.bed.gz
K562.TSS.selected_by_highest_Pol2_signal.bed.gz
loopB.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed.gz



In [39]:
FD_REG=${FD_RES}/results/region/hic_intact_ENCSR479XDG/loop
FD_ANT=${FD_RES}/results/region/annotation_tss_pol2
FD_OUT=${FD_RES}/results/region/hic_intact_ENCSR479XDG/loop/annotation_tss_pol2

for PREFIX in "loopA" "loopB"; do
    echo ${PREFIX}
    FN_REG=${PREFIX}.bed.gz
    FN_ANT=K562.TSS.selected_by_highest_Pol2_signal.bed.gz
    FN_OUT=${PREFIX}.annotation.${FN_ANT}
    
    FP_REG=${FD_REG}/${FN_REG}
    FP_ANT=${FD_ANT}/${FN_ANT}
    FP_OUT=${FD_OUT}/${FN_OUT}
    
    sbatch -p ${NODE} \
        --exclude=dl-01 \
        --cpus-per-task 8 \
        --mem 4G \
        --output ${FD_LOG}/annotation_hic_intact_ENCSR479XDG_${PREFIX}_tss_pol2.%a.txt \
        --array 0 \
        ./run_annot.sh ${FP_REG} ${FP_ANT} ${FP_OUT}
done

loopA
Submitted batch job 29825924
loopB
Submitted batch job 29825925


**Check Results**

In [40]:
cat ${FD_LOG}/annotation_hic_intact_ENCSR479XDG_loopA_tss_pol2.0.txt

Hostname:           x1-03-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         05-18-23+13:06:51

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/loopA.bed.gz

show first few lines of input
chr1	770000	780000
chr1	904000	906000
chr1	920000	925000
chr1	980000	990000
chr1	984000	985000

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_tss_pol2/K562.TSS.selected_by_highest_Pol2_signal.bed.gz

show first few lines of input
chr1	11873	11874	DDX11L1	2.3e-4
chr1	17436	17437	MIR6859-1	9.43812
chr1	17436	17437	MIR6859-2	9.43812
chr1	17436	17437	MIR6859-3	9.43812
chr1	17436	17437	MIR6859-4	9.43812


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/annotation_tss_pol2/loopA.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed.gz

show first few lines of output:
chr1	770000	780000	chr1	778634	778635	LOC100288069	230.561	1
chr1	980000	990000	chr1	

In [41]:
cat ${FD_LOG}/annotation_hic_intact_ENCSR479XDG_loopB_tss_pol2.0.txt

Hostname:           x1-03-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         05-18-23+13:06:51

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/loopB.bed.gz

show first few lines of input
chr1	840000	850000
chr1	976000	978000
chr1	1060000	1065000
chr1	1060000	1065000
chr1	1060000	1065000

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_tss_pol2/K562.TSS.selected_by_highest_Pol2_signal.bed.gz

show first few lines of input
chr1	11873	11874	DDX11L1	2.3e-4
chr1	17436	17437	MIR6859-1	9.43812
chr1	17436	17437	MIR6859-2	9.43812
chr1	17436	17437	MIR6859-3	9.43812
chr1	17436	17437	MIR6859-4	9.43812


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG/loop/annotation_tss_pol2/loopB.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed.gz

show first few lines of output:
chr1	1116000	1117000	chr1	1116089	1116090	C1orf159	16.4374	1
chr1	1205000	12100

In [15]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_tss_pol2.0.txt

Hostname:           x2-06-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         05-01-23+13:56:02

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_tss_pol2/K562.TSS.selected_by_highest_Pol2_signal.bed

show first few lines of input
chr19	58351969	58351970	A1BG-AS1	1.5218
chr19	58353492	58353493	A1BG	5.62284
chr10	50885675	50885676	A1CF	0.00706126
chr12	9065176	9065177	A2M-AS1	0.00201544
chr12	9115919	9115920	A2M	0.0400522


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_tss_pol2/peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed.gz

show first few lines of output:
chr

In [10]:
ls /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_tss_pol2/peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_tss_pol2/peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed


In [11]:
zcat /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/annotation_tss_pol2/peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed | head

chr1	17237	17772	chr1	17436	17437	MIR6859-1	9.43812	1
chr1	17237	17772	chr1	17436	17437	MIR6859-2	9.43812	1
chr1	17237	17772	chr1	17436	17437	MIR6859-3	9.43812	1
chr1	17237	17772	chr1	17436	17437	MIR6859-4	9.43812	1
chr1	28903	29613	chr1	29370	29371	WASH7P	2.3e-4	1
chr1	777949	779437	chr1	778634	778635	LOC100288069	230.561	1
chr1	816774	817547	chr1	817370	817371	FAM87B	1.06609	1
chr1	826754	828040	chr1	827522	827523	LINC00115	64.4656	1
chr1	826754	828040	chr1	827590	827591	LINC01128	64.4603	1
chr1	876474	878030	chr1	876802	876803	FAM41C	0.00788399	1
